# Examine a dynamic mixing matrix over time

Here we use model.graph to compute the dynamic mixing matrix values used in our model.  Note that we are not running the full model, merely using the portion of the graph we are interested in; graph.filter will ensure we are using the required parameters

In [ ]:
from autumn.core.project import get_project

In [ ]:
p = get_project("sm_covid2", "france")

In [ ]:
baseparams = p.param_set.baseline.to_dict()

In [ ]:
m = p.build_model(baseparams)

In [ ]:
defp = m.builder.get_default_parameters()

In [ ]:
from jax import jit
import numpy as np

In [ ]:
# Filter the graph so we are just obtaining the mixing matrix, no extraneous computation

mmgraph = m.graph.filter("mixing_matrix")
mmgraph.draw()

In [ ]:
# Get a callable function from this graph, and jit it for speed
mmfunc = jit(mmgraph.get_callable())

In [ ]:
# Examine the required inputs to our function
# The source names are the kwargs to our callable; these are dictionaries whose own keys are v.key
for v in mmgraph.get_input_variables():
    print(f"{v.source}: {v.key}")

In [ ]:
# Obtain stacked [time, agegroup, agegroup] mixing matrix data
# Use the source/key pairs as shown above
mm = np.stack([mmfunc(model_variables={"time": t}, parameters={"school_multiplier": 1.0})["mixing_matrix"] for t in m.times])
mm.shape

In [ ]:
from plotly import express as px

In [ ]:
px.imshow(mm, animation_frame=0, range_color=[0.0, 10.0])